# **EXTRACTING TEXT AND CREATING JSON FROM NEW FILE**

In [1]:
!pip install PyMuPDF
!pip install Pix2Text

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.7/215.7 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.6/224.6 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 711.2/711.2 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import fitz  # PyMuPDF
from pix2text import Pix2Text
import google.generativeai as genai

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
DATA_DIR = "/content/drive/My Drive/BDS_Topic_Clustering/final_jsons"
OUTPUT_DIR = "/content/drive/My Drive/BDS_Topic_Clustering/fresh_output_embeddings"
NEW_FILES_DIR = "/content/drive/My Drive/BDS_Topic_Clustering/new_pdf_files"

# **1. Convert PDFs to Markdowns**

In [ ]:
# code from pixImple.py

def run_pix2text(img_fp, page_number):
    p2t = Pix2Text.from_config(device="cpu")
    kwargs = {
        "resized_shape": 1280,
        "mfr_batch_size": 1,
        "embed_sep": (" $", "$ "),
        "isolated_sep": ("$$", "$$"),
        "line_sep": "\n",
        "auto_line_break": True,
        "det_text_bbox_max_width_expand_ratio": 0.3,
        "det_text_bbox_max_height_expand_ratio": 0.2,
        "embed_ratio_threshold": 0.6,
        "table_as_image": True,
        "formula_rec_kwargs": {}
    }
    try:
        page = p2t.recognize_page(img_fp, page_number=page_number, **kwargs)
        return page
    except Exception as e:
        print(f"Error processing page {page_number}: {e}")
        return None

def save_pix2text_output(page, page_number, output_page_dir):
    os.makedirs(output_page_dir, exist_ok=True)
    output_md_path = os.path.join(output_page_dir, f'page_{page_number}.md')
    try:
        page.to_markdown(output_md_path)
        print(f"Saved Markdown to {output_md_path}")
    except Exception as e:
        print(f"Error saving Markdown for page {page_number}: {e}")

    if hasattr(page, 'images') and page.images:
        for img_name, img_data in page.images.items():
            output_img_path = os.path.join(output_page_dir, f'{page_number}_img_{img_name}')
            try:
                with open(output_img_path, 'wb') as img_file:
                    img_file.write(img_data)
                print(f"Saved image to {output_img_path}")
            except Exception as e:
                print(f"Error saving image {img_name} for page {page_number}: {e}")

# modified from pixImple to process PDFs by each page into markdown files
def process_pdfs(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    pdf_files = [f for f in os.listdir(input_dir) if f.endswith('.pdf')]

    for pdf_file in pdf_files:
        pdf_path = os.path.join(input_dir, pdf_file)
        pdf_output_dir = os.path.join(output_dir, os.path.splitext(pdf_file)[0])
        os.makedirs(pdf_output_dir, exist_ok=True)

        doc = fitz.open(pdf_path)
        for page_number in range(len(doc)):
            page = doc.load_page(page_number)
            img_fp = os.path.join(pdf_output_dir, f'page_{page_number + 1}.png')
            pix = page.get_pixmap()
            pix.save(img_fp)

            result = run_pix2text(img_fp, page_number + 1)
            if result:
                save_pix2text_output(result, page_number + 1, pdf_output_dir)

        doc.close()
        print(f"Completed processing {pdf_file}")

# Process pdfs into output folder
input_dir = NEW_FILES_DIR
output_dir = "/content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds"
process_pdfs(input_dir, output_dir)


The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



densenet_lite_136-gru-onnx.zip:   0%|          | 0.00/11.6M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



0: 1024x736 1 title, 10 plain texts, 6 abandons, 1 table, 1 table_caption, 125.5ms
Speed: 33.7ms preprocess, 125.5ms inference, 335.1ms postprocess per image at shape (1, 3, 1024, 736)
Loading /root/.pix2text/1.1/mfd-onnx/mfd-v20240618.onnx for ONNX Runtime inference...
Using ONNX Runtime CPUExecutionProvider

WARNING ⚠️ imgsz=[198, 1280] must be multiple of max stride 32, updating to [224, 1280]
0: 224x1280 (no detections), 235.6ms
Speed: 2.8ms preprocess, 235.6ms inference, 8.2ms postprocess per image at shape (1, 3, 224, 1280)


0it [00:00, ?it/s]


...


WARNING ⚠️ imgsz=[407, 1280] must be multiple of max stride 32, updating to [416, 1280]
0: 416x1280 (no detections), 412.3ms
Speed: 3.8ms preprocess, 412.3ms inference, 0.6ms postprocess per image at shape (1, 3, 416, 1280)


0it [00:00, ?it/s]



WARNING ⚠️ imgsz=[1811, 1280] must be multiple of max stride 32, updating to [1824, 1280]
0: 1824x1280 (no detections), 2441.4ms
Speed: 17.4ms preprocess, 2441.4ms inference, 1.4ms postprocess per image at shape (1, 3, 1824, 1280)


0it [00:00, ?it/s]


Saved Markdown to /content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds/4788/page_1.md
Completed processing 4788.pdf


# **2. Combine Markdowns into One**

In [ ]:
# function to combine all markdowns within each pdf folder to output single markdown
def combine_output_markdowns(output_dir):
    for folder in os.listdir(output_dir):
        folder_path = os.path.join(output_dir, folder)
        if os.path.isdir(folder_path):
            combined_md_path = os.path.join(folder_path, f'{folder}_combined.md')
            with open(combined_md_path, 'w', encoding='utf-8') as combined_file:
                markdown_found = False
                # Traverse subdirectories to find output.md
                for subdir in os.listdir(folder_path):
                    subdir_path = os.path.join(folder_path, subdir)
                    if os.path.isdir(subdir_path):
                        output_md_path = os.path.join(subdir_path, 'output.md')
                        if os.path.isfile(output_md_path):
                            with open(output_md_path, 'r', encoding='utf-8') as md_file:
                                content = md_file.read().strip()
                                if content:
                                    combined_file.write(f'# {subdir}\n\n')  # Subdir name as header
                                    combined_file.write(content + '\n\n')
                                    markdown_found = True

                if not markdown_found:
                    print(f"No output.md files found in {folder_path}, combined file will be empty.")
                else:
                    print(f"Combined markdown saved to {combined_md_path}")


In [ ]:
# Combine markdowns
output_dir = "/content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds"
combine_output_markdowns(output_dir)

Combined markdown saved to /content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds/4788/4788_combined.md


# **3. Build JSON from Combined Markdown**

In [ ]:
from google.colab import userdata

In [ ]:
# Set your API key
genai.configure(api_key = userdata.get('GEMINI_KEY'))
# Load the JSON schema
with open("/content/drive/My Drive/BDS_Topic_Clustering/json_schema.txt", "r") as file:
    json_schema = file.read()

# Function to process a single Markdown file
def process_markdown_file(md_file_path):
    with open(md_file_path, "r") as file:
        markdown_content = file.read()

    # Prepare the prompt
    prompt = f"""
    Based on the following JSON schema, extract the information from the provided Markdown content and generate a JSON output:

    JSON Schema:
    {json_schema}

    Markdown Content:
    {markdown_content}

    Please produce a JSON output following the schema exactly.
    """

    # Call the Gemini API
    # model = genai.GenerativeModel('gemini-2.0-pro-exp') #or gemini-pro-vision if you need it.
    model = genai.GenerativeModel('gemini-2.0-flash') #or gemini-pro-vision if you need it.
    response = model.generate_content(prompt)

    # Get the generated JSON
    generated_json = response.text

    # Save JSON file in the same folder as the Markdown file
    output_file_path = os.path.splitext(md_file_path)[0] + ".json"
    with open(output_file_path, "w") as file:
        file.write(generated_json)

    print(f"JSON output saved to {output_file_path}")


In [ ]:
# Traverse multiple folders and process each Markdown file
root_dir = "/content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds"
target_subdirs = ["4788"]

for subdir in target_subdirs:
    subdir_path = os.path.join(root_dir, subdir)

    for file in os.listdir(subdir_path):
        if file.endswith(".md") and os.path.isfile(os.path.join(subdir_path, file)):
            md_file_path = os.path.join(subdir_path, file)

            # Check if the .md file is directly in the target subdirectory (not in a nested subdirectory)
            if os.path.dirname(md_file_path) == subdir_path:
                print(f"Processing {md_file_path}...")
                process_markdown_file(md_file_path)

print("Processing complete.")

Processing /content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds/4788/4788_combined.md...
JSON output saved to /content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds/4788/4788_combined.json
Processing complete.


# **CREATING EMBEDDINGS AND CLUSTERS FROM OLD FILES**

In [5]:
!pip install transformers
!pip install nltk
!pip install tqdm
!pip install faiss-cpu sentence-transformers
!pip install openai
!pip install google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 64.0 MB/s eta 0:00:00


In [6]:
import os
import openai
import google.generativeai as genai
import json
import numpy as np
import pandas as pd
import torch
import nltk
import faiss
import re
from collections import Counter
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

Link to All folders - https://drive.google.com/drive/folders/19jKtMboy7-_yJW4cfWFLAUzrvt-xJRzy?usp=sharing

In [15]:
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [17]:
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize components
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

# **1. Load JSON files and extract text**

In [ ]:
def load_json_files(data_dir):
    #Load JSON files and extract relevant text fields, handling missing values.
    all_texts = []
    file_ids = []

    for file in os.listdir(data_dir):
        if file.endswith(".json"):
            file_path = os.path.join(data_dir, file)
            with open(file_path, "r", encoding="utf-8") as f:
                data = json.load(f)

            # Extract text, handle missing values
            title = data["metadata"].get("title", "") or ""
            abstract = " ".join([para.get("text", "") for para in data["metadata"].get("abstract", []) if para.get("text")])
            body_text = " ".join([para.get("text", "") for para in data["metadata"].get("body_text", []) if para.get("text")])

            # Combine all extracted text
            combined_text = f"{title} {abstract} {body_text}".strip()

            if combined_text:  # Ensure non-empty entries
                file_ids.append(file.split(".json")[0])
                all_texts.append(combined_text)

    return file_ids, all_texts

# **2. Preprocess Text**

In [13]:
def preprocess_text(text):
    #Lowercase, tokenize, remove stopwords, and lemmatize.
    words = word_tokenize(text.lower())  # Lowercasing & tokenization
    words = [lemmatizer.lemmatize(word) for word in words if word.isalnum() and word not in stop_words]
    return " ".join(words)

# **3. Generate TF-IDF Embeddings**

In [ ]:
def generate_tfidf_embeddings(texts):
    #Compute TF-IDF embeddings.
    vectorizer = TfidfVectorizer(max_features=5000)  # Limit features for efficiency
    tfidf_matrix = vectorizer.fit_transform(texts)
    return tfidf_matrix.toarray(), vectorizer

# **4. Save embeddings**

In [ ]:
def save_embeddings(file_ids, tfidf_embeddings):
    #Save embeddings as .npy files and CSV summaries.
    np.save(os.path.join(OUTPUT_DIR, "tfidf_embeddings.npy"), tfidf_embeddings)

    # Save file mappings for reference
    df = pd.DataFrame({"file_id": file_ids})
    df.to_csv(os.path.join(OUTPUT_DIR, "file_mappings.csv"), index=False)

    print("Embeddings saved successfully!")

# **5. Run the pipeline**

In [ ]:
print("Loading JSON files...")
file_ids, raw_texts = load_json_files(DATA_DIR)

print("Preprocessing text...")
processed_texts = [preprocess_text(text) for text in tqdm(raw_texts, desc="Preprocessing")]

print("Generating TF-IDF embeddings...")
tfidf_embeddings, vectorizer = generate_tfidf_embeddings(processed_texts)

print("Saving embeddings...")
save_embeddings(file_ids, tfidf_embeddings)

print("All done!")

Loading JSON files...
Preprocessing text...


Preprocessing: 100%|██████████| 79/79 [00:04<00:00, 17.84it/s]


Generating TF-IDF embeddings...
Saving embeddings...
Embeddings saved successfully!
All done!


# **6. Saving the TF-IDF vectorizer for future use**

In [ ]:
# Save vectorizer
import pickle
with open(os.path.join(OUTPUT_DIR, "tfidf_vectorizer.pkl"), "wb") as f:
    pickle.dump(vectorizer, f)

# **7. Checking shapes of the embeddings (to ensure all files were processed)**

In [ ]:
# Load file mappings
df = pd.read_csv("/content/drive/My Drive/BDS_Topic_Clustering/output_embeddings/file_mappings.csv")

# Load embeddings
tfidf_embeddings = np.load("/content/drive/My Drive/BDS_Topic_Clustering/output_embeddings/tfidf_embeddings.npy")

# Check counts
print(f"Number of JSONs processed: {len(df)}")
print(f"TF-IDF Embeddings Shape: {tfidf_embeddings.shape}")

Number of JSONs processed: 79
TF-IDF Embeddings Shape: (79, 5000)


# **8. Class-based TF-IDF and Topic Extraction**

In [ ]:
tfidf_embeddings = np.load(os.path.join(OUTPUT_DIR, "tfidf_embeddings.npy"))
df_files = pd.read_csv(os.path.join(OUTPUT_DIR, "file_mappings.csv"))  # File mappings

print(f"TF-IDF Embeddings Shape: {tfidf_embeddings.shape}")
print(f"Number of documents: {len(df_files)}")

# Cluster Using K-Means
NUM_CLUSTERS = 5
kmeans = KMeans(n_clusters=NUM_CLUSTERS, random_state=42)
df_files["cluster"] = kmeans.fit_predict(tfidf_embeddings)

# Saving K-Means model for future use
with open(os.path.join(OUTPUT_DIR, "kmeans_model.pkl"), "wb") as f:
    pickle.dump(kmeans, f)

# Load JSONs and Extract Texts (Titles & Abstracts)
def load_texts(data_dir):
    texts = {}
    doc_titles = {}
    for file in os.listdir(data_dir):
        if file.endswith(".json"):
            file_id = file.replace(".json", "")  # Ensure file ID is a string
            with open(os.path.join(data_dir, file), "r", encoding="utf-8") as f:
                data = json.load(f)
            title = data["metadata"].get("title", "") or "Untitled Document"
            abstract = " ".join([para.get("text", "") for para in data["metadata"].get("abstract", []) if para.get("text")])
            body_text = " ".join([para.get("text", "") for para in data["metadata"].get("body_text", []) if para.get("text")])
            texts[file_id] = f"{title} {abstract} {body_text}".strip()
            doc_titles[file_id] = f"{title} - {abstract[:200]}..."
    return texts, doc_titles

print("Loading document texts...")
doc_texts, doc_titles = load_texts(DATA_DIR)

# Convert file_id in df_files to string
df_files["file_id"] = df_files["file_id"].astype(str)

# Merge texts within each cluster, ensuring no empty entries
cluster_texts = []
for i in range(NUM_CLUSTERS):
    texts_in_cluster = [doc_texts[file_id] for file_id in df_files[df_files["cluster"] == i]["file_id"] if file_id in doc_texts]
    merged_text = " ".join(texts_in_cluster).strip()

    # Apply preprocessing to remove stopwords from cluster text
    if merged_text:
        cluster_texts.append(preprocess_text(merged_text))
    else:
        cluster_texts.append("empty cluster")

# Compute TF-IDF for clusters
vectorizer = TfidfVectorizer(max_features=1000)
cluster_tfidf = vectorizer.fit_transform(cluster_texts)

# Get feature names (words)
feature_names = vectorizer.get_feature_names_out()

# Extract Top 6 Keywords Per Cluster
print("Extracting top keywords...")
top_keywords = {}
for i in range(NUM_CLUSTERS):
    sorted_indices = cluster_tfidf[i].toarray().argsort()[0][-6:][::-1]  # Get top 6 keywords
    top_keywords[i] = [feature_names[idx] for idx in sorted_indices]

# Find Representative Documents (Titles & Abstracts)
print("Finding representative documents...")
rep_docs = {}
for i in range(NUM_CLUSTERS):
    cluster_indices = df_files[df_files["cluster"] == i].index
    cluster_embeddings = tfidf_embeddings[cluster_indices]
    centroid = kmeans.cluster_centers_[i].reshape(1, -1)

    # Compute distances to centroid
    distances = euclidean_distances(cluster_embeddings, centroid).flatten()

    # Get indices of top 3 closest documents
    top_doc_indices = cluster_indices[np.argsort(distances)[:3]]
    rep_docs[i] = [doc_titles[df_files.loc[idx, "file_id"]] for idx in top_doc_indices]

# Results
print("\n**Cluster Summary**")
for cluster_id in range(NUM_CLUSTERS):
    print(f"\n Cluster {cluster_id}:")
    print("Representative Documents:")
    for doc in rep_docs[cluster_id]:
        print(f"   - {doc}\n")
    print(f"Top Keywords: {', '.join(top_keywords[cluster_id])}")

# Save results to CSV
df_summary = pd.DataFrame({
    "Cluster": list(range(NUM_CLUSTERS)),
    "Representative Documents": ["\n".join(rep_docs[i]) for i in range(NUM_CLUSTERS)],
    "Top Keywords": [", ".join(top_keywords[i]) for i in range(NUM_CLUSTERS)]
})
df_summary.to_csv(os.path.join(OUTPUT_DIR, "cluster_summary.csv"), index=False)

print("\nCluster summary saved as cluster_summary.csv")

TF-IDF Embeddings Shape: (79, 5000)
Number of documents: 79
Loading document texts...
Extracting top keywords...
Finding representative documents...

**Cluster Summary**

 Cluster 0:
Representative Documents:
   - Search for Multibody Nuclear Reactions in Metal Deuteride Induced with Ion Beam and Electrolysis Methods - We report here the experimental results suggesting the occurrence of multibody nuclear reactions in metal deuterides under ion-beam irradiation and electrolysis. A meaningful increase of
helium-4 was ...

   - PRODUCTION OF HIGH ENERGY CHARGED PARTICLES DURING DEUTERON IMPLANTATION OF TITANIUM DEUTERIDES - Implantation experiments using 300-keV deuteron beams are performed to study the 3-body reaction in metal deuterides with full use of in situ analyses of the target. The AE-E telescope and the angular...

Top Keywords: reaction, mev, spectrum, energy, target, deuteron

 Cluster 1:
Representative Documents:
   - PROGRESS TOWARDS REPLICATION - ...

   - Input to Theory f

# **9. Improving Topic Representations using Gemini**

In [ ]:
import time

SUMMARY_FILE = os.path.join(OUTPUT_DIR, "cluster_summary.csv")

# Load existing cluster summaries
df_summary = pd.read_csv(SUMMARY_FILE)

genai.configure(api_key = userdata.get('NEW_GEMINI_KEY'))

gemini_model = genai.GenerativeModel("gemini-1.5-pro")

def generate_topic_description_gemini(keywords, representative_docs):

    prompt = (
        f"Given the following information about documents (research papers) in a cluster:\n"
        f"Keywords: {', '.join(keywords)}\n"
        f"Representative Documents: {representative_docs}\n"
        f"Please provide a meaningful and concise topic description for this cluster."
    )

    try:
        response = gemini_model.generate_content(prompt)

        # Extract the response
        topic_description = response.text.strip()
        return topic_description

    except Exception as e:
        print(f"Error generating topic for: {keywords} → {e}")
        return "Error generating topic"

# Generate & Save Refined Topics
for index, row in df_summary.iterrows():
    time.sleep(10)
    refined_topic = generate_topic_description_gemini(row["Top Keywords"].split(", "), row["Representative Documents"])

    print(f"\n Cluster {row['Cluster']}: {refined_topic}\n")

    df_summary.at[index, "Refined Topic"] = refined_topic

# Save updated results
df_summary.to_csv(os.path.join(OUTPUT_DIR, "refined_cluster_summary.csv"), index=False)

print("\n Refined topics saved as refined_cluster_summary.csv")


 Cluster 0: Deuteron-Induced Nuclear Reactions in Metal Deuterides


 Cluster 1: Cold Fusion/Low Energy Nuclear Reactions (LENR) in Palladium


 Cluster 2: Electrolytic Cell Reactions and Excess Heat:  Studies on electrolysis with various electrolytes (e.g., LiOD, LiOH, NaF, K2CO3) and electrodes (e.g., Pt, Pd, Ni) focusing on excess heat production and potential fusion reactions.



Error generating topic for: ['energy', 'electron', 'fusion', 'deuteron', 'nucleus', 'pd'] → 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.

 Cluster 3: Error generating topic



Error generating topic for: ['patent', 'claim', 'fusion', 'cold', 'may', 'heat'] → 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-pro:generateContent?%24alt=json%3Benum-encoding%3Dint: You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits.

 Cluster 4: Error generating topic


 Refined topics saved as refined_cluster_summary.csv


In [ ]:
# Load existing file
df_summary = pd.read_csv(os.path.join(OUTPUT_DIR, "refined_cluster_summary.csv"))

# Loop over clusters with errors (due to time limits)
for index, row in df_summary.iterrows():
    if str(row["Refined Topic"]).startswith("Error"):
        print(f"\n⏳ Retrying Cluster {row['Cluster']}...")
        time.sleep(10)  # Wait to avoid rate limit exhaustion

        keywords = [kw.strip() for kw in row["Top Keywords"].split(",")]
        topic = generate_topic_description_gemini(keywords, row["Representative Documents"])

        df_summary.at[index, "Refined Topic"] = topic
        print(f" Cluster {row['Cluster']} updated: {topic}\n")

# Save back to the same file
df_summary.to_csv(os.path.join(OUTPUT_DIR, "refined_cluster_summary.csv"), index=False)
print("\n All updates saved to refined_cluster_summary.csv")


⏳ Retrying Cluster 3...
 Cluster 3 updated: **Cold Fusion in Metal Deuterides: Theoretical and Computational Studies**


⏳ Retrying Cluster 4...
 Cluster 4 updated: Cold Fusion Controversy and Research


 All updates saved to refined_cluster_summary.csv


Input Prompt:

"Given the following information about documents (research papers) in a cluster:
        Keywords: {top 6 keywords}
        Representative Documents: {closest 3 representative docs},
Please provide a meaningful and concise topic description for this cluster."

Refined Topic Representations:

* Cluster 0: Deuteron-Induced Nuclear Reactions in Metal Deuterides
* Cluster 1: Cold Fusion/Low Energy Nuclear Reactions (LENR) in Palladium
* Cluster 2: Electrolytic Cell Reactions and Excess Heat:  Studies on electrolysis with various electrolytes (e.g., LiOD, LiOH, NaF, K2CO3) and electrodes (e.g., Pt, Pd, Ni) focusing on excess heat production and potential fusion reactions.
* Cluster 3: Cold Fusion in Metal Deuterides: Theoretical and Computational Studies
* Cluster 4: Cold Fusion Controversy and Research

# **ASSIGNING NEW DOCUMENT TO AN EXISTING CLUSTER**

# **1. Load embedding vectorizer, cluster names, and K-Means model**

In [7]:
import pickle
import pandas as pd

VECTORIZER_PATH = os.path.join(OUTPUT_DIR, "tfidf_vectorizer.pkl")
CLUSTER_SUMMARY_PATH = os.path.join(OUTPUT_DIR, "refined_cluster_summary.csv")

# Load vectorizer
with open(VECTORIZER_PATH, "rb") as f:
    vectorizer = pickle.load(f)

# Load cluster summaries with names
df_cluster_summary = pd.read_csv(CLUSTER_SUMMARY_PATH)

# Load K-Means model
with open(os.path.join(OUTPUT_DIR, "kmeans_model.pkl"), "rb") as f:
    kmeans = pickle.load(f)

# **2. Load and preprocess new document (JSON)**

In [8]:
def load_and_preprocess_new_json(json_path):
    with open(json_path, "r", encoding="utf-8") as f:
        data = json.load(f)

    title = data["metadata"].get("title", "") or ""
    abstract = " ".join([para.get("text", "") for para in data["metadata"].get("abstract", []) if para.get("text")])
    body_text = " ".join([para.get("text", "") for para in data["metadata"].get("body_text", []) if para.get("text")])
    full_text = f"{title} {abstract} {body_text}".strip()

    if not full_text:
        raise ValueError("Document is empty.")

    return preprocess_text(full_text)

# **3. Vectorize new doc using TF-IDF**

In [9]:
def vectorize_text(text, vectorizer):
    return vectorizer.transform([text]).toarray()

# **4. Assign to cluster and report distances**

In [10]:
def assign_to_cluster(tfidf_vec, kmeans_model):
    from sklearn.metrics.pairwise import euclidean_distances
    distances = euclidean_distances(tfidf_vec, kmeans_model.cluster_centers_).flatten()
    closest_cluster = distances.argmin()
    return closest_cluster, distances

# **5. Output assigned cluster name and all distances**

In [11]:
def report_assignment(cluster_id, distances, df_cluster_summary):
    print("\n Distance to each cluster:")
    for i, dist in enumerate(distances):
        print(f"  Cluster {i}: {dist:.4f}")

    cluster_name = df_cluster_summary.loc[df_cluster_summary["Cluster"] == cluster_id, "Refined Topic"].values[0]
    print(f"\n Assigned to Cluster {cluster_id}: \"{cluster_name}\"")

# **6. Running the pipeline**

In [21]:
def assign_document_to_cluster(json_path, vectorizer, kmeans_model, df_cluster_summary, save_dir=None):
    print(f"\n Assigning document: {json_path}")
    text = load_and_preprocess_new_json(json_path)
    tfidf_vec = vectorize_text(text, vectorizer)
    cluster_id, distances = assign_to_cluster(tfidf_vec, kmeans_model)
    report_assignment(cluster_id, distances, df_cluster_summary)
    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        file_name = os.path.basename(json_path).replace(".json", "_embedding.npy")
        np.save(os.path.join(save_dir, file_name), tfidf_vec)
        print(f" Embedding saved to {os.path.join(save_dir, file_name)}")

assign_document_to_cluster(
    json_path="/content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds/4788/4788_combined.json",
    vectorizer=vectorizer,
    kmeans_model=kmeans,
    df_cluster_summary=df_cluster_summary,
    save_dir=os.path.join(OUTPUT_DIR, "new_doc_embedding")
)


 Assigning document: /content/drive/My Drive/BDS_Topic_Clustering/new_pdfs_mds/4788/4788_combined.json

 Distance to each cluster:
  Cluster 0: 1.2339
  Cluster 1: 0.9874
  Cluster 2: 1.0450
  Cluster 3: 1.0202
  Cluster 4: 1.0043

 Assigned to Cluster 1: "Cold Fusion/Low Energy Nuclear Reactions (LENR) in Palladium"
 Embedding saved to /content/drive/My Drive/BDS_Topic_Clustering/fresh_output_embeddings/new_doc_embedding/4788_combined_embedding.npy
